# Парсинг

In [3]:
import re
import json
import pandas as pd

In [28]:
with open('corpus.json') as json_file:
    dictionary = json.load(json_file)

In [29]:
values = list(dictionary.values())

In [37]:
for i in range(len(values)): # preprocess corpus if there are any excess symbols
  values[i][0] = re.sub('\\n', '', values[i][0])
  values[i][0] = re.sub('\\n\\n', '', values[i][0])
  values[i][0] = re.sub('\\r\\n', '', values[i][0])

In [5]:
from spacy_conll import init_parser
from spacy.symbols import ORTH
import spacy

nlp = init_parser("ru_core_news_sm", 'spacy', include_headers=True)

In [6]:
specials = ['мин.', 'ст. л.', 'ч. л.', '180°C']

In [38]:
link = 'https://www.gastronom.ru/recipe/'
for i in dictionary:
  current_link = f'{link}{i}'
  doc = nlp(dictionary[i][0])
  for spec in specials:
    if spec[:2] in doc.text:
      if spec[:-2] in doc.text:
        nlp.tokenizer.add_special_case(spec, [{ORTH: spec}])
  conll = doc._.conll_str
  conll = re.sub('([0-9]+)\t{5}SPACE\tSPACE.+\n', '', conll)
  conll = re.sub('\n\n', f'\n\n# source = {current_link}\n# name = {dictionary[i][1]}\n', conll)
  with open('corpus_parsed.conllu', 'a', encoding='utf-8') as f:
    f.write(f'\n# source = {current_link}\n# name = {dictionary[i][1]}\n')
    f.write(conll)